### Beginning of the Assignment - exploration

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
#pd.set_option('max_columns', 200)

### PHIL DATA SET

In [3]:
character_nicknames_df = pd.read_csv('datasets/character_nicknames.csv')

In [4]:
anime_details_df = pd.read_csv('datasets/details.csv')

In [5]:
favs_df = pd.read_csv('datasets/favs.csv')


In [6]:
person_details_df = pd.read_csv('datasets/person_details.csv')


In [7]:
person_alternate_names_df = pd.read_csv('datasets/person_alternate_names.csv')


In [8]:
person_anime_works_df = pd.read_csv('datasets/person_anime_works.csv')


In [9]:
stats_df = pd.read_csv('datasets/stats.csv')


### DENIS DATA SET

In [87]:
# converted high-cardinality string columns with many repeated values 
# (e.g. username) to the category type to reduce memory usage.
# This representation stores each distinct value once and references it via integer 
# codes, allowing the dataset to be processed efficiently on machines with limited RAM.

dtypes = {
    "is_rewatching": "Int8",
    "anime_id": "Int32",
    "score": "Int8",
    "num_watched_episodes": "Int32",
    "username": "category",
}
ratings_df = pd.read_csv("datasets/ratings.csv", dtype=dtypes)

In [11]:
characters_df = pd.read_csv('datasets/characters.csv')

In [12]:
character_anime_works_df = pd.read_csv('datasets/character_anime_works.csv')

In [3]:
person_voice_works_df = pd.read_csv('datasets/person_voice_works.csv')

In [14]:
profiles_df = pd.read_csv('datasets/profiles.csv')

In [15]:
recommendations_df = pd.read_csv('datasets/recommendations.csv')

# GUIDELINES
### BEFORE STARTING
Use Conda in order to do the correct setup

When we deliver the project, we need to tell the lecturer to run the jupyter notebook before running the TWEB part. How? Write it in the Report
Give instructions in order to make it run properly

#TODO to write on the report:
Which are the cool things, problems, map names not alined so we had to normalize them and so on

Things to do for each dataset:
1. Give it a look with .head and/or .tail
2. .describe and check if all the numeric values make sense (e.g. year=300 makes no sense in our context)
3. Check the format: objects to date if we need it. Check if all the dates are in the same format: us or eu
    also check if there's any 29/02/2013. It doesn't exist right? Maybe this is too much lol
4. Check for duplicates
5. CHECK FOR CORRELATION: df.corr() (e.g. with longer duration, there are more actors)
#TODO
6. ADD or Remove columns?
7. GROUPBY selects the elements and makes group out of it, combines the numeric fields of each specific group
#TODO We could use it grouping for language and looking at how many anime are made in japan, stating it's the first country where the culture of doing (and watching) anime is SO big
8. Aggregations: we can apply multiple different aggregated functions (e.g. for the first column you sum the data, for the second you do the average and so on)
9. Transformations: apply operations and return results aligned with the original DF

10. Removing NaNs is wrong in general because Pandas will skip it.
We do it when? Is it safe to remove NaNs rows if EVERY field in the row is empty? I hope so lol
BE CAREFUL if they are foreign keys: for example, if a person has a nan in the "anime he worked in" field, it shouldn't be dropped
NEVER replace with invalid values (e.g. -1)
IF we use df.dropna(subset=["name"],inplace=True)
the inplace means that the df itself is modified and will result in the one without the na. Without "inplace=true" you'll need to assign the result to another df (or the same) 


Proviamo i plot? df.plot()

11. Check if data are consistent (e.g. normalizing names of countries and/or numeric fields, describing them and checking what they are)

12. Normalize data types all in the same place (e.g. all the dates in the same cell)


BONUS: NEVER USE LOOP FOR, NEVER DUPLICATE DATA (unless necessary)


##### First look

In [16]:
character_nicknames_df

,character_mal_id,nickname
0,280205,Hikaruko
1,280129,Hinacchi
2,280127,Bertha Willis
3,280066,Jimmy
4,280059,Full Body Red Square
...,...,...
37075,282159,Ling Long
37076,282159,Silvermoon
37077,282227,Mei's Mother
37078,282254,Cyrano


In [17]:
character_nicknames_df.head()

,character_mal_id,nickname
0,280205,Hikaruko
1,280129,Hinacchi
2,280127,Bertha Willis
3,280066,Jimmy
4,280059,Full Body Red Square


In [18]:
character_nicknames_df.columns
#will list all the columns. Not necessary here but kept as a pattern to follow with the following files

Index(['character_mal_id', 'nickname'], dtype='object')

In [19]:
character_nicknames_df.dtypes

character_mal_id     int64
nickname            object
dtype: object

### Data cleaning

##### On the first dataset we may need to check for duplicates.
What does that mean? We have 102 rows that are duplicated over a 37080 rows dataset.
Why is that? Some characters have multiple nicknames, are they repeated in the dataset?

In [20]:
character_nicknames_df.describe(include='all')   

,character_mal_id,nickname
count,37080.000000,37064
unique,NaN,28928
top,NaN,Princess
freq,NaN,76
mean,115767.769579,NaN
std,87596.063426,NaN
min,3.000000,NaN
25%,30541.750000,NaN
50%,109656.000000,NaN
75%,192520.750000,NaN


In [21]:
character_nicknames_df.loc[character_nicknames_df.duplicated()]
#by default will give us the second

,character_mal_id,nickname
328,276501,Koko
329,276501,Cao Cao
352,275883,King
356,275835,Eldest Brother
447,274437,The Half-Fool
...,...,...
37051,281206,Apemon
37052,281206,Apemon
37053,281206,Apemon
37054,281206,Apemon


Mhh they're somehow different so yeah, the same character could have different nicknames.
We want to drop though the ones that are exactly the same.

In [22]:
#this way we drop the duplicates on the first dataset
character_nicknames_df.drop_duplicates(keep='first', inplace=True)

Let's check for nan values

In [23]:
character_nicknames_df[character_nicknames_df.isna().any(axis=1)]

,character_mal_id,nickname
696,271183,NaN
3170,248157,NaN
4880,230372,NaN
15899,135548,NaN
18598,107631,NaN
24278,48077,NaN
25041,42325,NaN
26604,35444,NaN
30115,19532,NaN
30116,19531,NaN


In [24]:
#cleaning the df from nan values
character_nicknames_df.dropna(inplace=True)


In [25]:
# OLD VERSION
# character_nicknames_df.to_csv("datasets/character_nicknames_cleaned.csv", index=False)
# With this command, the cleaned dataset is bigger than the original
# This is because on Windows line endings occupy 2 bytes instead of 1
# In order to fix it and have an actual smaller file that matches with macOS / Linux, we add the argument 
# lineterminator, which in older versions of pandas may not work, and must be replaced with line_terminator
character_nicknames_df.to_csv("datasets/character_nicknames_cleaned.csv", index=False, lineterminator="\n")

## SECOND DATASET

In [26]:
anime_details_df
#anime details
#japanes title could be dropped?
#members stand for how many users have this anime added to their list.
#explicit_genres is empty so can be removed
#licensor and streaming are mostly empty. Do we care?

,mal_id,title,title_japanese,url,image_url,type,status,score,scored_by,start_date,...,demographics,source,rating,episodes,season,year,producers,explicit_genres,licensors,streaming
0,59356,-Socket-,-socket-,https://myanimelist.net/anime/59356/-Socket-,https://cdn.myanimelist.net/images/anime/1043/...,Movie,Finished Airing,NaN,NaN,2010-01-01T00:00:00+00:00,...,[],Original,G - All Ages,1.0,NaN,NaN,['Nagoya Zokei University'],[],[],[]
1,56036,......,......,https://myanimelist.net/anime/56036/-,https://cdn.myanimelist.net/images/anime/1057/...,Music,Finished Airing,6.53,503.0,2023-06-11T00:00:00+00:00,...,[],Original,PG-13 - Teens 13 or older,1.0,NaN,NaN,[],[],[],[]
2,2928,.hack//G.U. Returner,.HACK//G.U. RETURNER,https://myanimelist.net/anime/2928/hack__GU_Re...,https://cdn.myanimelist.net/images/anime/1798/...,OVA,Finished Airing,6.65,9745.0,2007-01-18T00:00:00+00:00,...,[],Game,PG-13 - Teens 13 or older,1.0,NaN,NaN,"['Bandai Visual', 'CyberConnect2']",[],[],[]
3,3269,.hack//G.U. Trilogy,.hack//G.U. Trilogy,https://myanimelist.net/anime/3269/hack__GU_Tr...,https://cdn.myanimelist.net/images/anime/1566/...,Movie,Finished Airing,7.06,15373.0,2007-12-22T00:00:00+00:00,...,[],Game,PG-13 - Teens 13 or older,1.0,NaN,NaN,['Bandai Visual'],[],"['Funimation', 'Bandai Entertainment']",[]
4,4469,.hack//G.U. Trilogy: Parody Mode,.hack//G.U. Trilogy,https://myanimelist.net/anime/4469/hack__GU_Tr...,https://cdn.myanimelist.net/images/anime/10/86...,Special,Finished Airing,6.35,4317.0,2008-03-25T00:00:00+00:00,...,[],Game,PG-13 - Teens 13 or older,1.0,NaN,NaN,['Bandai Visual'],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28950,59421,Zutaboro Reijou wa Ane no Moto Konyakusha ni D...,ずたぼろ令嬢は姉の元婚約者に溺愛される,https://myanimelist.net/anime/59421/Zutaboro_R...,https://cdn.myanimelist.net/images/anime/1518/...,TV,Finished Airing,7.37,15624.0,2025-07-05T00:00:00+00:00,...,['Josei'],Light novel,PG-13 - Teens 13 or older,12.0,summer,2025.0,"['Studio Pierrot', 'Mainichi Broadcasting Syst...",[],[],"['Crunchyroll', 'Aniplus TV', 'Bahamut Anime C..."
28951,31245,Zutto Mae kara Suki deshita. Kokuhaku Jikkou I...,ずっと前から好きでした。～告白実行委員会～,https://myanimelist.net/anime/31245/Zutto_Mae_...,https://cdn.myanimelist.net/images/anime/3/821...,Movie,Finished Airing,7.20,104106.0,2016-04-23T00:00:00+00:00,...,[],Music,PG-13 - Teens 13 or older,1.0,NaN,NaN,"['Aniplex', 'Dentsu', 'Kadokawa Shoten', 'Movi...",[],['Aniplex of America'],[]
28952,36305,Zutto Mae kara Suki deshita. Kokuhaku Jikkou I...,ずっと前から好きでした。～告白実行委員会～ 「金曜日のおはよう」,https://myanimelist.net/anime/36305/Zutto_Mae_...,https://cdn.myanimelist.net/images/anime/6/883...,Special,Finished Airing,7.17,10038.0,2016-10-26T00:00:00+00:00,...,[],Music,PG - Children,1.0,NaN,NaN,['Aniplex'],[],[],[]
28953,34895,Zutto Suki Datta,ずっと好きだった,https://myanimelist.net/anime/34895/Zutto_Suki...,https://cdn.myanimelist.net/images/anime/1498/...,OVA,Finished Airing,5.68,1887.0,2017-04-21T00:00:00+00:00,...,[],Manga,Rx - Hentai,2.0,NaN,NaN,"['Queen Bee', 'Mediabank']",[],[],[]


In [27]:
anime_details_df.loc[anime_details_df.duplicated()]

,mal_id,title,title_japanese,url,image_url,type,status,score,scored_by,start_date,...,demographics,source,rating,episodes,season,year,producers,explicit_genres,licensors,streaming


There are no duplicates

In [28]:
anime_details_df['themes'].value_counts()
#seems to be no standard way of writing themes. Some are separated by commas, some by semicolons, some have extra spaces
#how can we standardize this?
#we can replace all semicolons with commas and then remove extra spaces

themes
[]                                                             11818
['Music']                                                       4015
['Anthropomorphic']                                              923
['School']                                                       912
['Historical']                                                   858
                                                               ...  
['Adult Cast', 'Survival']                                         1
['Adult Cast', 'Historical', 'Military', 'Time Travel']            1
['Adult Cast', 'Otaku Culture', 'Time Travel', 'Workplace']        1
['Historical', 'Mythology', 'Performing Arts']                     1
['Video Game', 'Villainess']                                       1
Name: count, Length: 1044, dtype: int64

In [29]:
# we want to see what are "type"
anime_details_df['type'].value_counts()

type
TV            8414
Movie         4915
OVA           4184
ONA           4096
Music         3999
Special       1755
TV Special     767
CM             483
PV             275
Name: count, dtype: int64

In [30]:
anime_details_df.query('year>2025')

,mal_id,title,title_japanese,url,image_url,type,status,score,scored_by,start_date,...,demographics,source,rating,episodes,season,year,producers,explicit_genres,licensors,streaming
134,61637,29-sai Dokushin Chuuken Boukensha no Nichijou,29歳独身中堅冒険者の日常,https://myanimelist.net/anime/61637/29-sai_Dok...,https://cdn.myanimelist.net/images/anime/1688/...,TV,Not yet aired,NaN,NaN,2026-01-01T00:00:00+00:00,...,['Shounen'],Manga,NaN,NaN,winter,2026.0,['Kadokawa'],[],[],[]
733,62000,Akuyaku Reijou wa Ringoku no Outaishi ni Dekia...,悪役令嬢は隣国の王太子に溺愛される,https://myanimelist.net/anime/62000/Akuyaku_Re...,https://cdn.myanimelist.net/images/anime/1383/...,TV,Not yet aired,NaN,NaN,2026-01-01T00:00:00+00:00,...,['Shoujo'],Light novel,NaN,NaN,winter,2026.0,['Kadokawa'],[],[],[]
1152,61333,Ao no Miburo: Serizawa Ansatsu-hen,青のミブロ 芹沢暗殺編,https://myanimelist.net/anime/61333/Ao_no_Mibu...,https://cdn.myanimelist.net/images/anime/1753/...,TV,Not yet aired,NaN,NaN,2025-12-20T00:00:00+00:00,...,['Shounen'],Manga,PG-13 - Teens 13 or older,NaN,winter,2026.0,[],[],[],[]
1355,60255,Arne no Jikenbo,アルネの事件簿,https://myanimelist.net/anime/60255/Arne_no_Ji...,https://cdn.myanimelist.net/images/anime/1289/...,TV,Not yet aired,NaN,NaN,2026-01-01T00:00:00+00:00,...,[],Game,NaN,12.0,winter,2026.0,['Asmik Ace'],[],[],[]
3361,60509,Champignon no Majo,シャンピニオンの魔女,https://myanimelist.net/anime/60509/Champignon...,https://cdn.myanimelist.net/images/anime/1135/...,TV,Not yet aired,NaN,NaN,2026-01-01T00:00:00+00:00,...,[],Manga,NaN,NaN,winter,2026.0,[],[],[],[]
4687,59853,Dark Moon: Tsuki no Saidan,DARK MOON -黒の月: 月の祭壇-,https://myanimelist.net/anime/59853/Dark_Moon_...,https://cdn.myanimelist.net/images/anime/1609/...,TV,Not yet aired,NaN,NaN,2026-01-01T00:00:00+00:00,...,[],Web manga,NaN,NaN,winter,2026.0,[],[],[],[]
4700,58886,Darwin Jihen,ダーウィン事変,https://myanimelist.net/anime/58886/Darwin_Jihen,https://cdn.myanimelist.net/images/anime/1740/...,TV,Not yet aired,NaN,NaN,2026-01-01T00:00:00+00:00,...,['Seinen'],Manga,NaN,NaN,winter,2026.0,['TOHO animation'],[],[],[]
4762,61196,Dead Account,デッドアカウント,https://myanimelist.net/anime/61196/Dead_Account,https://cdn.myanimelist.net/images/anime/1158/...,TV,Not yet aired,NaN,NaN,2026-01-01T00:00:00+00:00,...,['Shounen'],Manga,NaN,NaN,winter,2026.0,"['Nippon Columbia', 'Bushiroad']",[],[],[]
5459,61325,"Douse, Koishite Shimaunda. Season 2",どうせ、恋してしまうんだ。Season 2,https://myanimelist.net/anime/61325/Douse_Kois...,https://cdn.myanimelist.net/images/anime/1473/...,TV,Not yet aired,NaN,NaN,2026-01-01T00:00:00+00:00,...,['Shoujo'],Manga,PG-13 - Teens 13 or older,NaN,winter,2026.0,[],[],[],[]
5929,59229,Enen no Shouboutai: San no Shou Part 2,炎炎ノ消防隊 参ノ章 第2クール,https://myanimelist.net/anime/59229/Enen_no_Sh...,https://cdn.myanimelist.net/images/anime/1847/...,TV,Not yet aired,NaN,NaN,2026-01-01T00:00:00+00:00,...,['Shounen'],Manga,PG-13 - Teens 13 or older,NaN,winter,2026.0,[],[],[],[]


In [31]:
anime_details_df.describe()

,mal_id,score,scored_by,rank,popularity,members,favorites,episodes,year
count,28955.000000,18882.000000,1.888200e+04,21997.000000,28955.000000,2.895500e+04,28955.000000,28275.000000,6266.000000
mean,33977.521948,6.390500,2.996308e+04,11033.157794,14500.358798,3.875337e+04,430.848973,14.130080,2010.034153
std,19616.858566,0.892045,1.219666e+05,6388.279473,8373.663464,1.673763e+05,4520.610176,47.161445,13.200708
min,1.000000,1.890000,1.010000e+02,1.000000,1.000000,2.300000e+01,0.000000,1.000000,1961.000000
25%,15454.000000,5.770000,3.322500e+02,5505.000000,7247.500000,2.330000e+02,0.000000,1.000000,2005.000000
50%,37969.000000,6.360000,1.528000e+03,11008.000000,14497.000000,1.077000e+03,1.000000,2.000000,2014.000000
75%,50434.500000,7.030000,1.014575e+04,16515.000000,21754.500000,9.193500e+03,17.000000,13.000000,2020.000000
max,62590.000000,9.290000,2.979733e+06,22020.000000,28999.000000,4.230312e+06,243358.000000,3000.000000,2026.000000


In [32]:
anime_details_df[['start_date','season']].query("season.notna()")
#season can be removed? Do we care about the season? We could "calculate" it, if needed from the "start_date" field

,start_date,season
10,2006-04-06T00:00:00+00:00,spring
11,2002-04-04T00:00:00+00:00,spring
12,2003-01-09T00:00:00+00:00,winter
19,2024-07-07T00:00:00+00:00,summer
20,2025-01-05T00:00:00+00:00,winter
...,...,...
28906,2001-04-07T00:00:00+00:00,spring
28913,2011-05-18T00:00:00+00:00,spring
28932,1980-04-15T00:00:00+00:00,spring
28939,2012-04-03T00:00:00+00:00,spring


In [33]:
anime_details_df.drop(columns='season', inplace=True)

In [34]:
anime_details_df.query("episodes > 2500")
#checking outliers

,mal_id,title,title_japanese,url,image_url,type,status,score,scored_by,start_date,...,themes,demographics,source,rating,episodes,year,producers,explicit_genres,licensors,streaming
13697,9947,Lan Mao,蓝猫淘气3000问,https://myanimelist.net/anime/9947/Lan_Mao,https://cdn.myanimelist.net/images/anime/5/275...,TV,Finished Airing,6.06,228.0,1999-10-08T00:00:00+00:00,...,[],[],Original,PG - Children,3000.0,NaN,['Beijing Sunchime Happy Culture Company'],[],[],[]


In [35]:
anime_details_df[["start_date", "end_date"]] = anime_details_df[["start_date", "end_date"]].apply(
    pd.to_datetime, errors="coerce"
)

In [36]:
anime_details_df.query("start_date > end_date")

,mal_id,title,title_japanese,url,image_url,type,status,score,scored_by,start_date,...,themes,demographics,source,rating,episodes,year,producers,explicit_genres,licensors,streaming
125,48010,23 Hao Niu Nai Tang,23号牛乃糖,https://myanimelist.net/anime/48010/23_Hao_Niu...,https://cdn.myanimelist.net/images/anime/1789/...,TV,Finished Airing,NaN,NaN,2020-03-01 00:00:00+00:00,...,[],['Kids'],Original,PG - Children,26.0,NaN,[],[],[],[]
614,23583,Akage no Anne Specials,世界名作劇場・完結版 赤毛のアン,https://myanimelist.net/anime/23583/Akage_no_A...,https://cdn.myanimelist.net/images/anime/3/609...,TV Special,Finished Airing,6.25,355.0,2001-02-11 00:00:00+00:00,...,['Historical'],[],Novel,G - All Ages,2.0,NaN,['BS Fuji'],[],[],[]
755,44731,Ali Diu Dongxi de Wawa,阿狸·丢东西的娃娃,https://myanimelist.net/anime/44731/Ali_Diu_Do...,https://cdn.myanimelist.net/images/anime/1472/...,ONA,Finished Airing,NaN,NaN,2012-08-31 00:00:00+00:00,...,[],['Kids'],Picture book,PG-13 - Teens 13 or older,2.0,NaN,[],[],[],[]
1462,53727,Ashuai 8th Season,阿衰 第八季,https://myanimelist.net/anime/53727/Ashuai_8th...,https://cdn.myanimelist.net/images/anime/1982/...,ONA,Finished Airing,NaN,NaN,2022-06-03 00:00:00+00:00,...,['School'],['Kids'],Manga,G - All Ages,34.0,NaN,[],[],[],[]
1548,45385,AU: Kaixin Tongnian,阿U之开心童年,https://myanimelist.net/anime/45385/AU__Kaixin...,https://cdn.myanimelist.net/images/anime/1394/...,TV,Finished Airing,NaN,NaN,2013-03-01 00:00:00+00:00,...,[],['Kids'],Mixed media,PG - Children,60.0,NaN,[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28755,44012,Zhiqu Yang Xuetang: Yangyang Lai Yunbao,智趣羊学堂之羊羊来寻宝,https://myanimelist.net/anime/44012/Zhiqu_Yang...,https://cdn.myanimelist.net/images/anime/1439/...,ONA,Finished Airing,NaN,NaN,2018-02-22 00:00:00+00:00,...,[],['Kids'],Unknown,PG - Children,26.0,NaN,[],[],[],[]
28756,44010,Zhiqu Yang Xuetang: Yangyang You Shijie,智趣羊学堂之羊羊游世界,https://myanimelist.net/anime/44010/Zhiqu_Yang...,https://cdn.myanimelist.net/images/anime/1930/...,ONA,Finished Airing,NaN,NaN,2017-12-01 00:00:00+00:00,...,[],['Kids'],Unknown,PG - Children,26.0,NaN,[],[],[],[]
28830,38118,Zhu Zhu Xia: Mo Huan Zhu Luo Ji,猪猪侠 魔幻猪猡纪,https://myanimelist.net/anime/38118/Zhu_Zhu_Xi...,https://cdn.myanimelist.net/images/anime/1912/...,TV,Finished Airing,NaN,NaN,2006-07-01 00:00:00+00:00,...,"['Martial Arts', 'Super Power']",['Kids'],Original,G - All Ages,20.0,NaN,[],[],[],[]
28833,38119,Zhu Zhu Xia: Wu Xia 2008,猪猪侠 武侠2008,https://myanimelist.net/anime/38119/Zhu_Zhu_Xi...,https://cdn.myanimelist.net/images/anime/1658/...,TV,Finished Airing,NaN,NaN,2007-07-01 00:00:00+00:00,...,"['Historical', 'Super Power']",['Kids'],Original,G - All Ages,20.0,NaN,['BlueArc Animation Studios'],[],[],[]


In [37]:
anime_details_df.query("start_date > end_date and not (end_date.dt.month == 1 and end_date.dt.day == 1)")

,mal_id,title,title_japanese,url,image_url,type,status,score,scored_by,start_date,...,themes,demographics,source,rating,episodes,year,producers,explicit_genres,licensors,streaming
6392,3705,Flashback Game,フラッシュバックゲーム,https://myanimelist.net/anime/3705/Flashback_Game,https://cdn.myanimelist.net/images/anime/1870/...,OVA,Finished Airing,5.43,921.0,2001-10-18 00:00:00+00:00,...,[],[],Original,Rx - Hentai,3.0,NaN,['Five Ways'],[],['NuTech Digital'],[]
6971,10995,Ganbare!! Nattou-san,頑張れ!!納父さん,https://myanimelist.net/anime/10995/Ganbare_Na...,https://cdn.myanimelist.net/images/anime/9/744...,TV,Finished Airing,NaN,NaN,2011-07-02 00:00:00+00:00,...,[],[],Unknown,G - All Ages,4.0,2011.0,[],[],[],[]
12673,56892,Kkomimanyeo Lara Season 2,꼬미마녀 라라 시즌2,https://myanimelist.net/anime/56892/Kkomimanye...,https://cdn.myanimelist.net/images/anime/1348/...,TV,Finished Airing,NaN,NaN,2023-10-14 00:00:00+00:00,...,"['Mahou Shoujo', 'School']",['Kids'],Original,PG - Children,13.0,2023.0,[],[],[],[]
17729,34234,Ohayou! Kokekkou-san,おはよう! コケッコーさん,https://myanimelist.net/anime/34234/Ohayou_Kok...,https://cdn.myanimelist.net/images/anime/2/823...,TV,Finished Airing,NaN,NaN,2016-10-02 00:00:00+00:00,...,[],['Kids'],Original,G - All Ages,50.0,2016.0,['TMS Music'],[],[],[]
19590,43976,Qi Jiguang Yingxiong Chuan,戚继光英雄传,https://myanimelist.net/anime/43976/Qi_Jiguang...,https://cdn.myanimelist.net/images/anime/1831/...,Movie,Finished Airing,NaN,NaN,2012-06-01 00:00:00+00:00,...,['Historical'],['Kids'],Other,G - All Ages,1.0,NaN,[],[],[],[]
24814,38571,Tobidase! Dokan-kun,とびだせ！土管くん,https://myanimelist.net/anime/38571/Tobidase_D...,https://cdn.myanimelist.net/images/anime/1618/...,TV,Finished Airing,NaN,NaN,2011-10-26 00:00:00+00:00,...,['Mecha'],[],Original,G - All Ages,15.0,2011.0,[],[],[],[]
24837,62453,Tobot: Daedosiui Yeongungdeul Season 2 Part 2,또봇 : 대도시의 영웅들 시즌2 파트 2,https://myanimelist.net/anime/62453/Tobot__Dae...,https://cdn.myanimelist.net/images/anime/1721/...,TV,Finished Airing,NaN,NaN,2024-07-05 00:00:00+00:00,...,['Mecha'],['Kids'],Original,PG - Children,8.0,2024.0,[],[],[],[]
27477,61524,Xiongmao He Gan Mi Xiong,熊猫和甘米熊,https://myanimelist.net/anime/61524/Xiongmao_H...,https://cdn.myanimelist.net/images/anime/1216/...,TV,Finished Airing,NaN,NaN,2025-05-04 00:00:00+00:00,...,[],['Kids'],Other,PG - Children,13.0,2025.0,[],[],[],[]


In [38]:
anime_details_df.query("end_date.isna()")

,mal_id,title,title_japanese,url,image_url,type,status,score,scored_by,start_date,...,themes,demographics,source,rating,episodes,year,producers,explicit_genres,licensors,streaming
0,59356,-Socket-,-socket-,https://myanimelist.net/anime/59356/-Socket-,https://cdn.myanimelist.net/images/anime/1043/...,Movie,Finished Airing,NaN,NaN,2010-01-01 00:00:00+00:00,...,[],[],Original,G - All Ages,1.0,NaN,['Nagoya Zokei University'],[],[],[]
1,56036,......,......,https://myanimelist.net/anime/56036/-,https://cdn.myanimelist.net/images/anime/1057/...,Music,Finished Airing,6.53,503.0,2023-06-11 00:00:00+00:00,...,['Music'],[],Original,PG-13 - Teens 13 or older,1.0,NaN,[],[],[],[]
2,2928,.hack//G.U. Returner,.HACK//G.U. RETURNER,https://myanimelist.net/anime/2928/hack__GU_Re...,https://cdn.myanimelist.net/images/anime/1798/...,OVA,Finished Airing,6.65,9745.0,2007-01-18 00:00:00+00:00,...,['Video Game'],[],Game,PG-13 - Teens 13 or older,1.0,NaN,"['Bandai Visual', 'CyberConnect2']",[],[],[]
3,3269,.hack//G.U. Trilogy,.hack//G.U. Trilogy,https://myanimelist.net/anime/3269/hack__GU_Tr...,https://cdn.myanimelist.net/images/anime/1566/...,Movie,Finished Airing,7.06,15373.0,2007-12-22 00:00:00+00:00,...,['Video Game'],[],Game,PG-13 - Teens 13 or older,1.0,NaN,['Bandai Visual'],[],"['Funimation', 'Bandai Entertainment']",[]
4,4469,.hack//G.U. Trilogy: Parody Mode,.hack//G.U. Trilogy,https://myanimelist.net/anime/4469/hack__GU_Tr...,https://cdn.myanimelist.net/images/anime/10/86...,Special,Finished Airing,6.35,4317.0,2008-03-25 00:00:00+00:00,...,"['Parody', 'Video Game']",[],Game,PG-13 - Teens 13 or older,1.0,NaN,['Bandai Visual'],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28948,52512,Zurui Maboroshi,ズルい幻,https://myanimelist.net/anime/52512/Zurui_Mabo...,https://cdn.myanimelist.net/images/anime/1096/...,Music,Finished Airing,6.26,128.0,2022-02-22 00:00:00+00:00,...,['Music'],[],Original,PG-13 - Teens 13 or older,1.0,NaN,[],[],[],[]
28949,58303,Zurukute Sugoi,ズルくてすごい,https://myanimelist.net/anime/58303/Zurukute_S...,https://cdn.myanimelist.net/images/anime/1782/...,Music,Finished Airing,NaN,NaN,2019-08-17 00:00:00+00:00,...,['Music'],[],Original,PG-13 - Teens 13 or older,1.0,NaN,[],[],[],[]
28951,31245,Zutto Mae kara Suki deshita. Kokuhaku Jikkou I...,ずっと前から好きでした。～告白実行委員会～,https://myanimelist.net/anime/31245/Zutto_Mae_...,https://cdn.myanimelist.net/images/anime/3/821...,Movie,Finished Airing,7.20,104106.0,2016-04-23 00:00:00+00:00,...,['School'],[],Music,PG-13 - Teens 13 or older,1.0,NaN,"['Aniplex', 'Dentsu', 'Kadokawa Shoten', 'Movi...",[],['Aniplex of America'],[]
28952,36305,Zutto Mae kara Suki deshita. Kokuhaku Jikkou I...,ずっと前から好きでした。～告白実行委員会～ 「金曜日のおはよう」,https://myanimelist.net/anime/36305/Zutto_Mae_...,https://cdn.myanimelist.net/images/anime/6/883...,Special,Finished Airing,7.17,10038.0,2016-10-26 00:00:00+00:00,...,['School'],[],Music,PG - Children,1.0,NaN,['Aniplex'],[],[],[]


In [39]:
#Set invalid dates to NaT
#TODO put it in the report
anime_details_df.loc[anime_details_df["start_date"] > anime_details_df["end_date"], "end_date"] = pd.NaT


In [40]:
anime_details_df[["scored_by", "rank", "episodes", "year"]] = (
    anime_details_df[["scored_by", "rank", "episodes", "year"]].astype("Int64")
)


In [41]:
anime_details_df.dtypes
#scored_by, rank, episodes, year can be an int instead of a float
#start and end dates are not objects but dates
#do we need to swap the empty [] with Nan or not? WE should in order to be able to use the .isna() method and other pandas methods


mal_id                           int64
title                           object
title_japanese                  object
url                             object
image_url                       object
type                            object
status                          object
score                          float64
scored_by                        Int64
start_date         datetime64[ns, UTC]
end_date           datetime64[ns, UTC]
synopsis                        object
rank                             Int64
popularity                       int64
members                          int64
favorites                        int64
genres                          object
studios                         object
themes                          object
demographics                    object
source                          object
rating                          object
episodes                         Int64
year                             Int64
producers                       object
explicit_genres          

### THIRD DATASET

In [42]:
favs_df

,username,fav_type,id
0,ishikawas,anime,45649
1,ishikawas,anime,38680
2,ishikawas,anime,795
3,ishikawas,anime,37510
4,ishikawas,anime,820
...,...,...,...
4178742,vincent0607,character,497
4178743,vincent0607,character,118739
4178744,vincent0607,character,188177
4178745,vincent0607,character,141354


In [43]:
# we want to see what are "fav_type"
favs_df['fav_type'].value_counts()

fav_type
character    1598040
anime        1531857
people        862186
company       186664
Name: count, dtype: int64

In [44]:
favs_df.isna().sum()


username    4
fav_type    0
id          0
dtype: int64

In [45]:
favs_df.dtypes

username    object
fav_type    object
id           int64
dtype: object

In [46]:
favs_df.duplicated().sum()
#there are no duplicates


np.int64(0)

### FOURTH DATASET

In [47]:
person_alternate_names_df

,person_mal_id,alt_name
0,1,Seki Mondoya
1,1,門戸 開
2,1,Monto Hiraku
3,3,雪野五月
4,10,Kevin Hatcher
...,...,...
20460,89567,Sydsnap
20461,89567,Queen of Degeneracy
20462,89826,陳浩
20463,89842,Chidori


In [48]:
person_alternate_names_df.dtypes

person_mal_id     int64
alt_name         object
dtype: object

In [49]:
person_alternate_names_df['person_mal_id'].value_counts()
# the person with the most alternate names has 29 different names

person_mal_id
246      29
548      28
406      16
10957    16
1886     15
         ..
3         1
53        1
19        1
89179     1
58        1
Name: count, Length: 12376, dtype: int64

In [50]:
person_alternate_names_df['alt_name'].value_counts()
# 

alt_name
Friendly Land     6
アイス               6
Aice5             6
Aice⁵             5
Studio Wallaby    4
                 ..
堀江 由子             1
Hocchan           1
KAmiYU            1
Miyu-Miyu         1
Nobu              1
Name: count, Length: 20248, dtype: int64

In [51]:
person_alternate_names_df.isna().sum()
person_alternate_names_df[person_alternate_names_df.isna().any(axis=1)]


,person_mal_id,alt_name
1470,2813,NaN
1662,3380,NaN
2552,7070,NaN
3932,12406,NaN
7829,40897,NaN
8647,44952,NaN
8895,46425,NaN
9899,49829,NaN
11160,53940,NaN
11350,54493,NaN


In [52]:
person_alternate_names_df.dropna(inplace=True)



In [53]:
# person_alternate_names_df.loc[person_alternate_names_df['person_mal_id'].duplicated()]
person_alternate_names_df[person_alternate_names_df.duplicated(subset=['person_mal_id','alt_name'], keep=False)].sort_values(['person_mal_id','alt_name'])


,person_mal_id,alt_name
2526,7025,Fumio Tada
2531,7025,Fumio Tada
2536,7025,Fumio Tada
2528,7025,Ichirou Miyoshi
2533,7025,Ichirou Miyoshi
...,...,...
17383,77491,Joshua Ricardo Rocha Jiménez
17386,77514,Scott Page
17387,77514,Scott Page
18080,80213,Samantha Carmichael


In [54]:
person_alternate_names_df.drop_duplicates(keep='first', inplace=True)

### FIFTH DATASET

In [55]:
person_details_df

,person_mal_id,url,website_url,image_url,name,given_name,family_name,birthday,favorites,relevant_location
0,1,https://myanimelist.net/people/1/Tomokazu_Seki,NaN,https://cdn.myanimelist.net/images/voiceactors...,Tomokazu Seki,智一,関,1972-09-08T00:00:00+00:00,6219,"Berlin, Germany"
1,2,https://myanimelist.net/people/2/Tomokazu_Sugita,https://agrs.co.jp/,https://cdn.myanimelist.net/images/voiceactors...,Tomokazu Sugita,智和,杉田,1980-10-11T00:00:00+00:00,47666,"Los Angeles, USA"
2,3,https://myanimelist.net/people/3/Satsuki_Yukino,NaN,https://cdn.myanimelist.net/images/voiceactors...,Satsuki Yukino,さつき,ゆきの,1970-05-25T00:00:00+00:00,1777,"Madrid, Spain"
3,4,https://myanimelist.net/people/4/Aya_Hirano,http://ayahirano.jp/,https://cdn.myanimelist.net/images/voiceactors...,Aya Hirano,綾,平野,1987-10-08T00:00:00+00:00,18374,"Paris, France"
4,5,https://myanimelist.net/people/5/Kenichi_Suzumura,https://intention-k.com,https://cdn.myanimelist.net/images/voiceactors...,Kenichi Suzumura,健一,鈴村,1974-09-12T00:00:00+00:00,5176,"Osaka, Japan"
...,...,...,...,...,...,...,...,...,...,...
76694,90011,https://myanimelist.net/people/90011/Nanako_Ki...,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,Nanako Kishimoto,七子,岸本,NaN,0,"Mumbai, India"
76695,90012,https://myanimelist.net/people/90012/Pamon,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,Pamon,NaN,파몬,NaN,0,"Tokyo, Japan"
76696,90013,https://myanimelist.net/people/90013/Tomoru_Emoto,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,Tomoru Emoto,ともる,柄本,NaN,0,"Tokyo, Japan"
76697,90014,https://myanimelist.net/people/90014/Hirari,https://hirari.2-d.jp/,https://cdn.myanimelist.net/img/sp/icon/apple-...,Hirari,NaN,ひらり,NaN,0,"Paris, France"


In [56]:
person_details_df.loc[person_details_df['person_mal_id'].duplicated()]
#we found that the duplicates differ for the "relevant_location" field, which has no interest for us so we drop the duplicates

,person_mal_id,url,website_url,image_url,name,given_name,family_name,birthday,favorites,relevant_location
4171,4564,https://myanimelist.net/people/4564/Natsuko_Ta...,NaN,https://cdn.myanimelist.net/images/voiceactors...,Natsuko Takahashi,ナツコ,高橋,NaN,19,"Paris, France"
4172,4564,https://myanimelist.net/people/4564/Natsuko_Ta...,NaN,https://cdn.myanimelist.net/images/voiceactors...,Natsuko Takahashi,ナツコ,高橋,NaN,19,"Cape Town, South Africa"
5202,5742,https://myanimelist.net/people/5742/Sarah_Strange,NaN,https://cdn.myanimelist.net/images/voiceactors...,Sarah Strange,NaN,NaN,NaN,11,"Yokohama, Japan"
6018,6638,https://myanimelist.net/people/6638/Toshio_Suzuki,NaN,https://cdn.myanimelist.net/images/voiceactors...,Toshio Suzuki,敏夫,鈴木,1948-08-19T00:00:00+00:00,354,"Paris, France"
6019,6638,https://myanimelist.net/people/6638/Toshio_Suzuki,NaN,https://cdn.myanimelist.net/images/voiceactors...,Toshio Suzuki,敏夫,鈴木,1948-08-19T00:00:00+00:00,354,"Mumbai, India"
...,...,...,...,...,...,...,...,...,...,...
64387,77565,https://myanimelist.net/people/77565/Ena_Nishi...,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,Ena Nishikawa,絵奈,西川,NaN,0,"Paris, France"
64402,77579,https://myanimelist.net/people/77579/Amina_Gaede,https://www.instagram.com/amina.gaede/,https://cdn.myanimelist.net/images/voiceactors...,Amina Gaede,NaN,NaN,1998-07-05T00:00:00+00:00,0,"São Paulo, Brazil"
64412,77588,https://myanimelist.net/people/77588/Willie_Ra...,NaN,https://cdn.myanimelist.net/images/voiceactors...,Willie Ray Jr.,NaN,NaN,NaN,0,"Berlin, Germany"
64414,77589,https://myanimelist.net/people/77589/Brandi_Ray,NaN,https://cdn.myanimelist.net/images/voiceactors...,Brandi Ray,NaN,NaN,NaN,0,"Mumbai, India"


In [57]:
person_details_df.drop_duplicates(subset=['person_mal_id', 'url', 'name'], keep='first', inplace=True)


In [58]:
person_details_df.dtypes
#we need to change birthday from object to data

person_mal_id         int64
url                  object
website_url          object
image_url            object
name                 object
given_name           object
family_name          object
birthday             object
favorites             int64
relevant_location    object
dtype: object

In [59]:
person_details_df["birthday"] = pd.to_datetime(person_details_df["birthday"], errors='coerce')

In [60]:
person_details_df["birthday"].min(), person_details_df["birthday"].max()
#makes sense because they're just composers of used music in anime

(Timestamp('1678-03-04 00:00:00+0000', tz='UTC'),
 Timestamp('2023-06-15 00:00:00+0000', tz='UTC'))

In [61]:
person_details_df.isna().sum()
#we have to check the nan values

person_mal_id            0
url                      0
website_url          59186
image_url                0
name                     2
given_name           30211
family_name          18610
birthday             59399
favorites                0
relevant_location        0
dtype: int64

In [62]:
person_details_df[person_details_df["name"].isna()]


,person_mal_id,url,website_url,image_url,name,given_name,family_name,birthday,favorites,relevant_location
46687,59857,https://myanimelist.net/people/59857/None,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,NaN,NaN,のね,NaT,0,"Paris, France"
75920,89106,https://myanimelist.net/people/89106/NULL,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,NaN,NaN,NaN,NaT,0,"Paris, France"


We can join the two tables person_details_df and person_alternate_names_df having the keys that match.
Putting the alternate names in a new column called alt_name and having a list of those inside

### SIXTH DATASET

In [63]:
person_anime_works_df

,person_mal_id,position,anime_mal_id
0,1,Theme Song Performance,3080
1,1,Inserted Song Performance,15699
2,1,Theme Song Performance (OP),247
3,1,Theme Song Performance,258
4,1,Theme Song Performance (ED),34825
...,...,...,...
458086,89951,In-Between Animation,11001
458087,89951,Key Animation,55092
458088,89951,"Key Animation (eps 4, 9)",20053
458089,89951,Key Animation,50553


In [64]:
person_anime_works_df.dtypes
#the types are correct

person_mal_id     int64
position         object
anime_mal_id      int64
dtype: object

In [65]:
person_anime_works_df.isna().sum()
#There's no nan value

person_mal_id    0
position         0
anime_mal_id     0
dtype: int64

### SEVENTH DATASET

In [66]:
stats_df.filter(regex="_votes$").astype("Int64")
stats_df[stats_df.filter(regex="_votes$").columns] = (
    stats_df.filter(regex="_votes$").astype("Int64")
)

In [67]:
stats_df.isna().sum()
#there are 430 series without any votes

mal_id                   0
watching                 0
completed                0
on_hold                  0
dropped                  0
plan_to_watch            0
total                    0
score_1_votes          430
score_1_percentage     430
score_2_votes          430
score_2_percentage     430
score_3_votes          430
score_3_percentage     430
score_4_votes          430
score_4_percentage     430
score_5_votes          430
score_5_percentage     430
score_6_votes          430
score_6_percentage     430
score_7_votes          430
score_7_percentage     430
score_8_votes          430
score_8_percentage     430
score_9_votes          430
score_9_percentage     430
score_10_votes         430
score_10_percentage    430
dtype: int64

### EIGHTH DATASET 

In [68]:
ratings_df

,username,anime_id,status,score,is_rewatching,num_watched_episodes
0,--------788,30276,watching,7,0.0,3
1,--------788,28851,completed,7,0.0,1
2,--------788,41168,completed,7,0.0,1
3,--------788,22199,completed,10,0.0,24
4,--------788,16498,completed,10,0.0,25
...,...,...,...,...,...,...
124298352,arizkim,52305,plan_to_watch,0,0.0,0
124298353,arizkim,4224,plan_to_watch,0,0.0,0
124298354,arizkim,54790,plan_to_watch,0,0.0,0
124298355,arizkim,53835,plan_to_watch,0,0.0,0


In [69]:
ratings_df.columns

Index(['username', 'anime_id', 'status', 'score', 'is_rewatching',
       'num_watched_episodes'],
      dtype='object')

In [70]:
# we have to understand the sense of "num_watched_episodes" and the link with "is_rewatching"
ratings_df.query('is_rewatching == 1')

,username,anime_id,status,score,is_rewatching,num_watched_episodes
934,CKK2,6,completed,9,1.0,8
8939,FollowYourHeart,1210,completed,10,1.0,24
11047,----Haku----,39017,completed,7,1.0,3
22879,KarioBaka,10087,completed,9,1.0,3
22928,KarioBaka,14467,completed,0,1.0,6
...,...,...,...,...,...,...
124291819,arissabelle,2167,completed,0,1.0,0
124291878,arissabelle,33352,completed,10,1.0,0
124291883,arissabelle,6547,on_hold,0,1.0,1
124297299,ariyanroy04,46569,completed,0,1.0,13


In [89]:
ratings_df.dtypes

username                category
anime_id                   Int32
status                    object
score                       Int8
is_rewatching               Int8
num_watched_episodes       Int32
dtype: object

In [73]:
ratings_df[["anime_id","score","num_watched_episodes"]].agg(["min", "max"])


,anime_id,score,num_watched_episodes
min,1,0,0
max,62893,10,65535


In [77]:
ratings_df[ratings_df.duplicated(subset=['username','anime_id'], keep=False)].sort_values(['username','anime_id'])

,username,anime_id,status,score,is_rewatching,num_watched_episodes
74586609,Doopelsdoo,59459,watching,0,0,8
74616569,Doopelsdoo,59459,watching,0,0,9
74594917,Door_mp3,59644,watching,8,0,5
74626628,Door_mp3,59644,watching,8,0,6
74594935,Door_mp3,60564,watching,8,0,4
74626629,Door_mp3,60564,watching,8,0,5
74594941,Door_mp3,61159,watching,9,0,4
74626630,Door_mp3,61159,watching,9,0,5
74566332,Pavle2009,44511,watching,0,0,5
74614909,Pavle2009,44511,watching,0,0,7


In [90]:
# usually we should drop all the occurrence of a duplicate and keep the first
# in this case though, it looks like the latest occurence is the most updated one, contaning more info than the first one, so we drop the first one
ratings_df.drop_duplicates(subset=['username', 'anime_id'], keep='last', inplace=True)

We had just 6 duplicates having the same username and anime_id

In [102]:
# we check for Nan values.
ratings_df.isna().sum()

username                          0
anime_id                          0
status                            0
score                             0
is_rewatching                     0
num_watched_episodes              0
total_episodes              2352051
watched_more_than_exists          0
dtype: int64

In [ ]:
# We fill the NaN values in "is_rewatching" checking if the num_watched_episodes is greater than number of episodes of the anime itself
episodes_map = anime_details_df.set_index("mal_id")["episodes"]

ratings_df["total_episodes"] = ratings_df["anime_id"].map(episodes_map)

valid_mask = (ratings_df["total_episodes"].notna() & (ratings_df["total_episodes"] > 0) &
              ratings_df["num_watched_episodes"].notna() & (ratings_df["num_watched_episodes"] >= 0))

condition = ((ratings_df["num_watched_episodes"] > ratings_df["total_episodes"]) &valid_mask)
ratings_df["is_rewatching"] = ratings_df["is_rewatching"].fillna(condition)

To compare num_watched_episodes with the total number of episodes per anime without duplicating a 4 GB table in memory, we used a mapping from anime_id to episodes instead of a full merge. This approach is significantly lighter on RAM and therefore more suitable for lower RAM machines.

In [ ]:
ratings_df[ratings_df['username'].isna()]
# there's only one username with Nan value, we drop it because it's not relevant in our analysis

,username,anime_id,status,score,is_rewatching,num_watched_episodes


In [98]:
ratings_df.dropna(subset=["username"],inplace=True)

In [103]:
# first_chunk = True
# for chunk in pd.read_csv("datasets/ratings.csv", chunksize=2_000_000):
#    chunk["is_rewatching"] = chunk["is_rewatching"].astype("Int8")
#    chunk["anime_id"] = chunk["anime_id"].astype("Int32")
#    chunk["score"] = chunk["score"].astype("Int8")
#    chunk["num_watched_episodes"] = chunk["num_watched_episodes"].astype("Int32")
#
#    chunk.to_csv(
#        "datasets/ratings_half_cleaned.csv", mode="w" if first_chunk else "a",
#        index=False, header=first_chunk,lineterminator="\n" 
#    )
#    first_chunk = False

# ratings_cleaned_df = pd.read_csv("datasets/ratings_half_cleaned.csv")
# ratings_cleaned_df.drop_duplicates(subset=["username", "anime_id"], keep="last", inplace=True)
# ratings_cleaned_df.to_csv("datasets/ratings_cleaned.csv", index=False, lineterminator="\n")

This takes sooo long I'm not sure it's worth it. More than 12 minutes on my most powerful machine.
The cleaned version is about the same size as the original one and we save just a bit of memory when loading it (10% less)
This is because the file will be read and written twice, just for a small gain in memory usage
Why twice? Because if we drop duplicates whithin each chunk while reading it, we may miss duplicates that are in different chunks.

In [ ]:
ratings.to_csv("datasets/ratings_cleaned.csv", index=False, lineterminator="\n")
ratings_df.dropna(subset=["username"],inplace=True)

### NINTH DATASET

In [ ]:
characters_df

In [ ]:
# check types of dataset columns
characters_df.dtypes

In [ ]:
characters_df.describe()

In [ ]:
# we have only 2 rows where all columns are Nan, the rows with Nan values in "name_kanji" and "about" we shouldn't drop because they have other values that are important.
characters_df.isna().sum()

In [ ]:
# here we want to check if the Nan values are concentrate in only two rows
characters_df[characters_df['character_mal_id'].isna()]

In [ ]:
characters_df[characters_df['name_kanji'].isna()]

In [ ]:
# Apart "name_kanji" and "about" the others Nan values are concentrate in two rows so we drop the two rows with all columns Nan
characters_df.dropna(how='all', inplace=True)


In [ ]:
# we want to see all duplicates to understand if we have to drop or not
characters_df.loc[characters_df.duplicated(subset=['character_mal_id', 'url', 'name'], keep=False)]

In [ ]:
# we drop the duplicates because they have all same values 
characters_df.drop_duplicates(subset=['character_mal_id', 'url', 'name'], keep='first', inplace=True)

### TENTH DATASET

In [104]:
# role of character anime
character_anime_works_df

,anime_mal_id,character_mal_id,character_name,role
0,2928,5781,Atoli,Main
1,2928,33,Haseo,Main
2,2928,32,Ovan,Main
3,2928,34,Shino,Main
4,2928,5785,Aina,Supporting
...,...,...,...,...
236811,31245,137157,"Shibasaki, Ken",Supporting
236812,36305,136064,"Hamanaka, Midori",Main
236813,36305,133916,"Narumi, Sena",Main
236814,36305,124942,"Hayasaka, Akari",Supporting


In [107]:
# check types of columns
character_anime_works_df.dtypes

anime_mal_id         int64
character_mal_id     int64
character_name      object
role                object
dtype: object

In [106]:
# check the number of Nan value
character_anime_works_df.isna().sum()

anime_mal_id        0
character_mal_id    0
character_name      0
role                0
dtype: int64

In [105]:
# check the number of duplicates
character_anime_works_df.loc[character_anime_works_df.duplicated(subset=['anime_mal_id', 'character_mal_id'])]

,anime_mal_id,character_mal_id,character_name,role


There is no need to clean this dataset 

### ELEVENTH DATASET

In [108]:
person_voice_works_df

,person_mal_id,role,anime_mal_id,character_mal_id,language
0,1,Main,55830,2514,Japanese
1,1,Supporting,60602,2822,Japanese
2,1,Supporting,59229,140499,Japanese
3,1,Supporting,60427,275856,Japanese
4,1,Supporting,62067,190335,Japanese
...,...,...,...,...,...
489511,89839,Supporting,40111,274622,Mandarin
489512,89840,Supporting,60544,266412,Mandarin
489513,89841,Supporting,60544,266416,Mandarin
489514,89842,Supporting,36896,279922,Japanese


In [109]:
person_voice_works_df['language'].value_counts()

language
Japanese           203537
English             94902
French              43340
Spanish             39153
Portuguese (BR)     37100
Italian             30712
German              26805
Korean               7981
Mandarin             3529
Hungarian            1378
Hebrew               1079
Name: count, dtype: int64

In [ ]:
person_voice_works_df['language'].sum()

In [ ]:
#I want to group by language and see the different languages available in the dataset.
person_voice_works_df.groupby('language').size()

In [ ]:
person_voice_works_df.dtypes

In [ ]:
person_voice_works_df.isna().sum()

In [ ]:
# check if the duplicates are in all columns
person_voice_works_df.loc[person_voice_works_df.duplicated(keep=False)]

In [ ]:
# drop the duplicates because they have all same values
person_voice_works_df.drop_duplicates(keep='first', inplace=True)

### TWELFTH DATASET

In [ ]:
# Should we delete the last five columns?
# TODO
profiles_df

In [ ]:
# check if the types are right for each field
profiles_df.dtypes

In [ ]:
# trying things with the date
# BEFORE running the .to_datetime command, ishiyama_yumi has Jul 24 bday
profiles_df.loc[44]

In [ ]:
# change types of columns "birthday" and "joined" from object to date and the others columns that they should be int
profiles_df["birthday"] = pd.to_datetime(profiles_df["birthday"], errors='coerce')
profiles_df["joined"] = pd.to_datetime(profiles_df["joined"], errors='coerce')

In [ ]:
# AFTER running the .to_datetime command, ishiyama_yumi has NaT birthday
profiles_df.loc[44]

In [ ]:
profiles_df["birthday"].min(), profiles_df["birthday"].max()


In [ ]:
weird_birthdays = profiles_df[
    (profiles_df["birthday"] < "1900-01-01") |
    (profiles_df["birthday"] > "2025-12-31")
]

weird_birthdays


In [ ]:
#TODO
# Just noticed some birthdays have no year but just day and month, how could we manage those?
# I guess with errors='coerce', when we change type from object to date, the year is set to a default one (e.g. 1800 or 1900)

In [ ]:
profiles_df.loc[
    profiles_df["birthday"] > profiles_df["joined"],
    ["birthday", "joined"]
]


In [ ]:
profiles_df["joined"].min(), profiles_df["joined"].max()

In [ ]:
profiles_df["birthday"].dt.year.value_counts().sort_index().head(30)

In [ ]:
profiles_df[profiles_df["birthday"].dt.year == 1930]

In [ ]:
# decided to remove the birthdays of people older than 100 when they joined the website
mask_too_old = (profiles_df["joined"] - profiles_df["birthday"]).dt.days / 365.25 > 100
profiles_df.loc[mask_too_old, "birthday"] = pd.NaT


In [ ]:
mask_too_young = (profiles_df["joined"] - profiles_df["birthday"]).dt.days / 365.25 < 3
mask_too_young


In [ ]:
profiles_df.isna().sum()
#TODO one guy has null username
# We'll put a progressive unkown_user[] label on it, so that for future data with missing username we could go on with it

In [ ]:
# check if there is any duplicate on "username"
profiles_df.loc[profiles_df.duplicated(subset=['username'], keep='first')]
# none found

### THIRTEENTH DATASET

In [ ]:
recommendations_df

In [ ]:
recommendations_df.dtypes

In [ ]:
recommendations_df.isna().sum()

In [ ]:
recommendations_df.loc[profiles_df.duplicated(keep='first')]

# TO KEEP

In [ ]:
# 1st
# dropping duplicates
character_nicknames_df.drop_duplicates(keep='first', inplace=True)
# dropping nan values
character_nicknames_df.dropna(inplace=True)

# 2nd
# no need to clean from duplicates nor missing values
# change column types from object to datetime
anime_details_df[["start_date", "end_date"]] = anime_details_df[["start_date", "end_date"]].apply(
    pd.to_datetime, errors="coerce"
)
# change column types from object to Int64
anime_details_df[["scored_by", "rank", "episodes", "year"]] = (
    anime_details_df[["scored_by", "rank", "episodes", "year"]].astype("Int64")
)

# 3rd
# no need to clean


# 4th
#dropping nan values because they don't give any useful information
person_alternate_names_df.dropna(inplace=True)
# dropping duplicates
person_alternate_names_df.drop_duplicates(keep='first', inplace=True)


# 5th
# Dropping duplicates choosing to keep the first occurrence. They only differ for the "relevant_location" field which has no interest for us
person_details_df.drop_duplicates(subset=['person_mal_id', 'url', 'name'], keep='first', inplace=True)
# change column types from object to datetime
person_details_df["birthday"] = pd.to_datetime(person_details_df["birthday"], errors='coerce')

# There are two weird rows with nan values in most columns but we don't drop them because they may be linked to other datasets
# Could we join the two tables person_details_df and person_alternate_names_df having the keys that match.
# Putting the alternate names in a new column called alt_name and having a list of those inside
# TODO


# 6th
# no need to clean


# 7th
# change column types to save memory
stats_df.filter(regex="_votes$").astype("Int64")
stats_df[stats_df.filter(regex="_votes$").columns] = (
    stats_df.filter(regex="_votes$").astype("Int64")
)


# 8th
# change column types to save memory
ratings_df["is_rewatching"] = ratings_df["is_rewatching"].astype("Int8")
ratings_df["anime_id"] = ratings_df["anime_id"].astype("Int32")
ratings_df["score"] = ratings_df["score"].astype("Int8")
ratings_df["num_watched_episodes"] = ratings_df["num_watched_episodes"].astype("Int32")
# drop duplicates keeping the last entry (most recent)
ratings_df.drop_duplicates(subset=['username', 'anime_id'], keep='last', inplace=True)
# there's a username with NaN value, we'll keep it for now as it matches with the profiles_df
# we only have 1 Nan value in "username" in the profiles_df so we can keep it for now
# I could manage it this way: generate a deterministic placeholder, e.g.:
# the maximum existing user_id + 1
# or a specific labeled ID like "unknown_user"
# THEN we can use this placeholder consistently across all datasets to maintain referential integrity.
# THIS won't work though because user_id is missing in both datasets, how can we be sure that the Nan in profiles_df matches the Nan in ratings_df?
# In this case it's easy because there's just one Nan in both datasets, but for future occasions we should do something else?
# 
# profiles_df["user_id"] = profiles_df["user_id"].fillna(new_id)
# ratings_df["user_id"] = ratings_df["user_id"].fillna(new_id)
# TODO

for chunk in pd.read_csv("datasets/ratings.csv", chunksize=1_000_000):
    chunk["is_rewatching"] = chunk["is_rewatching"].astype("Int8")
    chunk["anime_id"] = chunk["anime_id"].astype("Int32")
    chunk["score"] = chunk["score"].astype("Int8")
    chunk["num_watched_episodes"] = chunk["num_watched_episodes"].astype("Int32")
    
    chunk.to_csv(
        "datasets/ratings_cleaned.csv", mode="w" if first_chunk else "a", 
        index=False, header=first_chunk,lineterminator="\n"
    )
    first_chunk = False

ratings_cleaned_df = pd.read_csv("datasets/ratings_cleaned.csv")
ratings_cleaned_df.drop_duplicates(subset=["username", "anime_id"], keep="last", inplace=True)
ratings_cleaned_df.to_csv("datasets/ratings_cleaned.csv", index=False, lineterminator="\n")


# 9th
# change "character_mal_id" and "favorites" from float to int
characters_df[["character_mal_id", "favorites"]] = (
    characters_df[["character_mal_id", "favorites"]].astype("Int64")
)

# we drop the two rows with all columns Nan
characters_df.dropna(how='all', inplace=True)
# we drop the duplicates because they have all same values 
characters_df.drop_duplicates(subset=['character_mal_id', 'url', 'name'], keep='first', inplace=True)


# 10th
# no need to clean


# 11th
person_voice_works_df.drop_duplicates(keep='first', inplace=True)


# 12th
# change column types from object to datetime
profiles_df[["birthday", "joined"]] = profiles_df[["birthday", "joined"]].apply(
    pd.to_datetime, errors="coerce"
)
# setting birthdays outside a reasonable range to NaT
profiles_df.loc[
    profiles_df["birthday"] > profiles_df["joined"],
    "birthday"
] = pd.NaT
# decided to remove the birthdays of people older than 100 at join
mask_too_old = (profiles_df["joined"] - profiles_df["birthday"]).dt.days / 365.25 > 100
profiles_df.loc[mask_too_old, "birthday"] = pd.NaT
#I mean MongoDB, a female born in 1930-07-09, from Thailand and joined in 2017-07-09 makes perfect sense, doesn't it?

# 13th
# no need to clean


In [ ]:
#Exporting all cleaned small datasets
small_datasets = {
    "character_anime_works": character_anime_works_df,
    "character_nicknames": character_nicknames_df,
    "characters": characters_df,
    "anime_details": anime_details_df,
    "favs": favs_df,
    "person_alternate_names": person_alternate_names_df,
    "person_anime_works": person_anime_works_df,
    "person_details": person_details_df,
    "person_voice_works": person_voice_works_df,
    "profiles": profiles_df,
    #"ratings": ratings_df,
    "recommendations": recommendations_df,
    "stats": stats_df,
}

for name, df in small_datasets.items():
    df.to_csv(f"{name}_cleaned.csv", index=False, lineterminator="\n")


#TODO read it and apply
If the file is large due to repeated non-numeric data or unwanted columns, we can sometimes see massive memory savings by reading in columns as categories and selecting required columns via pd.read_csv usecols parameter.

### QUESTIONS:
1. Can we just leave the commands to clean the dataset instead of leaving all the exploration commands such as .describe, dtypes and so on
One file with data cleaning, showing the reasoning.
ANOTHER file with Data Visualization.

2. Ask for the maximum age in the 5th dataset
Justify why you have changed the dates (e.g. change 2070 in 1970) or nan. Possiamo fare il cazzo che vogliamo

3. Ask for the only NaN username in the "ratings" dataset: for the hci part?
Think in terms of the future, take the latest id and sum 1, or progressive uknownuserssssssss (e.g. uknown1, unknown2 in caso poi riceviamo altri dati nel futuro)

For TWEB we could just put a random (?) value in order not to lose the line and have non nan values in the database

SE NON USIAMO COLONNE IN ASSOLUTO, per alcuna query o nel sito tweb
Siccome non faremo mai query su questa cosa qui POSSIAMO TOGLIERE le colonne (e.g. kanji name lo possiamo togliere)